# Notebook configuration

In [33]:
# Configuration
shapefile_dir_path = '/home/william/Programming/WazeTT/data/input/bd_topo/ZONE_VEGETATION/'
shapefile_name = 'ZONE_VEGETATION.shp'
shapefile_path = shapefile_dir_path + shapefile_name

tiles_dir_path = '../data/input/bd_alti/'
tiles_name = 'dalles.shp'
tiles_path = tiles_dir_path + tiles_name

ascii_file_dir_path = '../data/input/bd_alti/'
ascii_file_name = 'BDALTIV2_25M_FXX_0825_6550_MNT_LAMB93_IGN69.asc'

# Load the ascii file in memory

In [34]:
with open(ascii_file_dir_path + ascii_file_name, 'r') as ascii_file:
    ncols = int(ascii_file.readline().replace('ncols', '').replace(' ', '').replace('\n', ''))
    nrows = int(ascii_file.readline().replace('nrows', '').replace(' ', '').replace('\n', ''))
    xllcorner_lamb93 = float(ascii_file.readline().replace('xllcorner', '').replace(' ', '').replace('\n', ''))
    yllcorner_lamb93 = float(ascii_file.readline().replace('yllcorner', '').replace(' ', '').replace('\n', ''))
    cellsize = int(float(ascii_file.readline().replace('cellsize', '').replace(' ', '').replace('\n', '')))
    NODATA_value = float(ascii_file.readline().replace('NODATA_value', '').replace(' ', '').replace('\n', ''))
    ascii_table = []
    for line in ascii_file.readlines():
        ascii_line = line.split(' ')
        ascii_line.remove('')
        ascii_table.append([float(cell) for cell in ascii_line])
    print(xllcorner_lamb93)

824987.5


# Deduce its Bounding Box coordinates in Lamb93 (xll, yll)

In [35]:
ascii_table_west_bound_xll_lamb93 = xllcorner_lamb93
ascii_table_east_bound_xll_lamb93 = xllcorner_lamb93 + cellsize*ncols
ascii_table_south_bound_yll_lamb93 = yllcorner_lamb93 - cellsize*nrows
ascii_table_north_bound_yll_lamb93 = yllcorner_lamb93

ascii_table_bb_coord_lamb93 = (
                                ascii_table_west_bound_xll_lamb93,
                                ascii_table_east_bound_xll_lamb93,
                                ascii_table_south_bound_yll_lamb93,
                                ascii_table_north_bound_yll_lamb93
                            )

print('ascii_table_west_bound_xll_lamb93 : {}'.format(ascii_table_west_bound_xll_lamb93))
print('ascii_table_east_bound_xll_lamb93 : {}'.format(ascii_table_east_bound_xll_lamb93))
print('ascii_table_south_bound_yll_lamb93 : {}'.format(ascii_table_south_bound_yll_lamb93))
print('ascii_table_north_bound_yll_lamb93 : {}'.format(ascii_table_north_bound_yll_lamb93))

ascii_table_west_bound_xll_lamb93 : 824987.5
ascii_table_east_bound_xll_lamb93 : 849987.5
ascii_table_south_bound_yll_lamb93 : 6500012.5
ascii_table_north_bound_yll_lamb93 : 6525012.5


# Create ASCII table Bounding Box polygon in Lamb93

In [36]:
ascii_table_nw_bound_coord_lamb93 = (ascii_table_west_bound_xll_lamb93, ascii_table_north_bound_yll_lamb93)
ascii_table_ne_bound_coord_lamb93 = (ascii_table_east_bound_xll_lamb93, ascii_table_north_bound_yll_lamb93)
ascii_table_se_bound_coord_lamb93 = (ascii_table_west_bound_xll_lamb93, ascii_table_south_bound_yll_lamb93)
ascii_table_sw_bound_coord_lamb93 = (ascii_table_west_bound_xll_lamb93, ascii_table_south_bound_yll_lamb93)

from shapely.geometry import Polygon

ascii_table_bb_polygon_lamb93 = Polygon([
                                    ascii_table_nw_bound_coord_lamb93,
                                    ascii_table_ne_bound_coord_lamb93,
                                    ascii_table_se_bound_coord_lamb93,
                                    ascii_table_sw_bound_coord_lamb93
                                ])
print(ascii_table_bb_polygon_lamb93)

POLYGON ((824987.5 6525012.5, 849987.5 6525012.5, 824987.5 6500012.5, 824987.5 6500012.5, 824987.5 6525012.5))


# Load the shapefile

In [40]:
# Conversion of a pair of files (.shp + .shx) into a list of POLYGON included into the ASCII table Bounding Box
import geopandas as gpd

gdf = gpd.read_file(
    shapefile_path,
    bbox=ascii_table_bb_coord_lamb93,
)

shapefile_polygones_wgs84 = gdf['geometry']
print(shapefile_polygones_wgs84[0])
# TODO : visualize these data with follium

POLYGON ((907748 6520508.1, 907742.9 6520504.1, 907739.3 6520505.7, 907737.6 6520510.6, 907739 6520516, 907742.8 6520518.3, 907748.2 6520515, 907748 6520508.1))


# (Not useful anymore)
Convert shapefile polygons coordinates from WGS84 to Lamb93

In [41]:
# from pyproj import Transformer
# lamb93_epsg_code = "2154" # The Lamb93 coordinates system -> https://epsg.io/2154
# wgs84_epsg_code = "4326" # The WGS84 coordinates system -> https://epsg.io/4326
# transformer = Transformer.from_crs("epsg:"+wgs84_epsg_code, "epsg:"+lamb93_epsg_code)

# for shapefile_polygon_wgs84 in shapefile_polygones_wgs84:
#     ascii_table_nw_bound_coord_wgs84 = transformer.transform(
#                                                                 ascii_table_nw_bound_coord_lamb93[0],
#                                                                 ascii_table_nw_bound_coord_lamb93[1]
#                                                             )
#     ascii_table_ne_bound_coord_wgs84 = transformer.transform(
#                                                                 ascii_table_ne_bound_coord_lamb93[0],
#                                                                 ascii_table_ne_bound_coord_lamb93[1]
#                                                             )
#     ascii_table_se_bound_coord_wgs84 = transformer.transform(
#                                                                 ascii_table_se_bound_coord_lamb93[0],
#                                                                 ascii_table_se_bound_coord_lamb93[1]
#                                                             )
#     ascii_table_sw_bound_coord_wgs84 = transformer.transform(
#                                                                 ascii_table_sw_bound_coord_lamb93[0],
#                                                                 ascii_table_sw_bound_coord_lamb93[1]
#                                                             )

# print('ascii_table_nw_bound_coord_wgs84 : {}'.format(ascii_table_nw_bound_coord_wgs84))
# print('ascii_table_ne_bound_coord_wgs84 : {}'.format(ascii_table_ne_bound_coord_wgs84))
# print('ascii_table_se_bound_coord_wgs84 : {}'.format(ascii_table_se_bound_coord_wgs84))
# print('ascii_table_sw_bound_coord_wgs84 : {}'.format(ascii_table_sw_bound_coord_wgs84))

# # TODO : visualize these data with follium

ascii_table_nw_bound_coord_wgs84 : (inf, inf)
ascii_table_ne_bound_coord_wgs84 : (inf, inf)
ascii_table_se_bound_coord_wgs84 : (inf, inf)
ascii_table_sw_bound_coord_wgs84 : (inf, inf)


# (Not useful anymore)
Project the Polygons extracted from the shapefile onto the 2D ASCII table (William)

In [21]:
# from shapely.geometry import Polygon

# projected_polygons = []
# for shapefile_polygon_wgs84 in shapefile_polygones_wgs84:
#     projected_polygon = ascii_table_bb_polygon_wgs84.intersection(shapefile_polygon_wgs84)
#     if projected_polygon.__str__() != 'POLYGON EMPTY':
#         print(projected_polygon)
#         projected_polygons.append(projected_polygon)

POLYGON ((839712.9 6575376.1, 839697.1 6575359.4, 839697.6 6575362.5, 839699.6 6575366.5, 839704.7 6575369.4, 839710.7 6575375.4, 839712.9 6575376.1))
POLYGON ((45.81328010871762 4.609623049975772, 45.80823268954793 4.93146214529326, 45.58820553013228 4.603087618767242, 45.58820553013228 4.603087618767242, 45.81328010871762 4.609623049975772))


KeyboardInterrupt: 

# Convert the polygons into 2D ASCII Tables (Bertrand)

# Merge all the tiles information into a single data structure (Jérémy)

# (Optionnal) Get shapefile BoundingBox coordinates
WGS84 coordinates system (lat, long)

In [36]:
# xml_path = gmd:MD_Metadata/gmd:identificationInfo/gmd:MD_DataIdentification/gmd:extent/gmd:EX_Extent/gmd:geographicElement/gmd:EX_GeographicBoundingBox
import xml.etree.ElementTree as ET
metadata_file_name = 'metadonnees.xml'
tree = ET.parse(shapefile_dir_path + metadata_file_name)
root = tree.getroot()

namespaces = {
                'gmd': '{http://www.isotc211.org/2005/gmd}',
                'gco': '{http://www.isotc211.org/2005/gco}'
            } # add more as needed

EX_GeographicBoundingBox_element = list(root.iter(namespaces['gmd']+'EX_GeographicBoundingBox'))[0]

west_bound_longitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'westBoundLongitude'))[0]
west_bound_longitude_decimal_element = list(west_bound_longitude_element.iter(namespaces['gco']+'Decimal'))[0]
west_bound_longitude_decimal_value = list(west_bound_longitude_decimal_element.itertext())[0]
print('west_bound_longitude_decimal_value : {}'.format(west_bound_longitude_decimal_value))

east_bound_longitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'eastBoundLongitude'))[0]
east_bound_longitude_decimal_element = list(east_bound_longitude_element.iter(namespaces['gco']+'Decimal'))[0]
east_bound_longitude_decimal_value = list(east_bound_longitude_decimal_element.itertext())[0]
print('east_bound_longitude_decimal_value : {}'.format(east_bound_longitude_decimal_value))

south_bound_latitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'southBoundLatitude'))[0]
south_bound_latitude_decimal_element = list(south_bound_latitude_element.iter(namespaces['gco']+'Decimal'))[0]
south_bound_latitude_decimal_value = list(south_bound_latitude_decimal_element.itertext())[0]
print('south_bound_latitude_decimal_value : {}'.format(south_bound_latitude_decimal_value))

north_bound_latitude_element = list(EX_GeographicBoundingBox_element.iter(namespaces['gmd']+'northBoundLatitude'))[0]
north_bound_latitude_decimal_element = list(north_bound_latitude_element.iter(namespaces['gco']+'Decimal'))[0]
north_bound_latitude_decimal_value = list(north_bound_latitude_decimal_element.itertext())[0]
print('north_bound_latitude_decimal_value : {}'.format(north_bound_latitude_decimal_value))

west_bound_longitude_decimal_value : 2.0629
east_bound_longitude_decimal_value : 7.1856
south_bound_latitude_decimal_value : 44.1155
north_bound_latitude_decimal_value : 46.8043
